In [8]:
#import nltk

In [9]:
#nltk.download()
#import nltk
#nltk.download('punkt')

In [10]:
import pandas as pd

In [11]:
# read the data into a pandas dataframe
import os
def data2df (path, label):
    file, text = [], []
    for f in os.listdir(path):
        file.append(f)
        fhr = open(path+f, 'r',encoding='utf-8', errors='ignore') 
        t = fhr.read()
        text.append(t)
        fhr.close()
    return(pd.DataFrame({'file': file, 'text': text, 'class':label}))

dfneg = data2df('HealthProNonPro/NonPro/', 0) # NonPro
dfpos = data2df('HealthProNonPro/Pro/', 1) # Pro

df = pd.concat([dfpos, dfneg], axis=0)
df.sample(frac=0.005)

,file,text,class
1500,a69635.txt,Dont worry! I have this same problem. I just f...,0
1864,ans990.txt,Possibly due to the hit on your head you are e...,1
1670,ans815.txt,Its unfortunate that the GP was not able to se...,1
566,a24798.txt,Try having and orgasm before having actual sex...,0
1442,ans61.txt,"According to your description, you may be expe...",1
1734,a7520.txt,"You need to get a humidifier, or put a pot of ...",0
212,a24444.txt,Find something soft and punch the crap outta i...,0
987,a61286.txt,"I have seen my husband break boards, and cinde...",0
1270,ans455.txt,I do understand your concern for your daughter...,1
1837,ans966.txt,Pain in the upper abdomen can be caused by a n...,1


In [12]:
# setup the data
X, y = df['text'], df['class']
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=1)

In [13]:
# setup a custom preprocessor
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk import word_tokenize
def preprocess(text):
    # replace one or more white-space characters with a space
    regex = re.compile(r"\s+")                               
    text = regex.sub(' ', text)    
    # lower case
    text = text.lower()          
    # remove digits and punctuation
    regex = re.compile(r"[%s%s]" % (string.punctuation, string.digits))
    text = regex.sub(' ', text)           
    # remove stop words
    sw = stopwords.words('english')
    text = text.split()                                              
    text = ' '.join([w for w in text if w not in sw]) 
    # remove short words
    ' '.join([w for w in text.split() if len(w) >= 2])
    # lemmatize
    text = ' '.join([(WordNetLemmatizer()).lemmatize(w) for w in text.split()]) 
    return text

In [14]:
# setup the preprocessing->model pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
clf = Pipeline(steps=[
    ('pp', TfidfVectorizer(
        preprocessor=preprocess,
        use_idf=True, smooth_idf=True,
        min_df=1, max_df=1.0, max_features=None, 
        ngram_range=(1, 1))),
    ('mdl',     MultinomialNB())
    #('mdl',     RandomForestClassifier())
    ])

In [15]:
# setup grid search
from sklearn.model_selection import GridSearchCV
param_grid = {
    'pp__norm':['l1','l2', None],
    'mdl__alpha':[0.01, 0.1, 0.2, 0.5, 1]
    
}
gscv = GridSearchCV(clf, param_grid, iid=False, cv=5, return_train_score=False)

In [16]:
# search for best params
gscv.fit(Xtrain, ytrain)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pp', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...se_idf=True, vocabulary=None)), ('mdl', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'pp__norm': ['l1', 'l2', None], 'mdl__alpha': [0.01, 0.1, 0.2, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [17]:
# evaluate best_estimator_ on test data
ypred = gscv.best_estimator_.predict(Xtest)
from sklearn import metrics
print (metrics.accuracy_score(ytest, ypred))
print (metrics.confusion_matrix(ytest, ypred))
print (metrics.classification_report(ytest, ypred))

0.9499545040946314
[[480  50]
 [  5 564]]
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       530
           1       0.92      0.99      0.95       569

   micro avg       0.95      0.95      0.95      1099
   macro avg       0.95      0.95      0.95      1099
weighted avg       0.95      0.95      0.95      1099

